# Annex : integration with loops instead of cumtrapz

In lesson 07 we used ``cumtrapz`` to compute the streamfunction: 

$$ \Psi (\varphi, p) = \frac{2 \pi R \cos \varphi}{g} \int_{p_{trop}}^{p} [v(\varphi, p)] dp $$

This example shows that it is possible to use for loops to achieve the same results. While I think that the cumtrap method is more elegant, the loops might help to undertand better what's going on.

In [ ]:
# Display the plots in the notebook:
%matplotlib inline
# Import the tools we are going to need today:
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xarray as xr  # netCDF library
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list
# Some defaults:
plt.rcParams['figure.figsize'] = (12, 5)  # Default plot size
np.set_printoptions(threshold=20)  # avoid to print very large arrays on screen

## For loops in python 

Until today we have not used a single for loop for our exercises. This is because numpy is made to work with multi-dimensional arrays without having to apply operations over array elements. For today's lesson however we will need one for loop, and you will also need them in your future work if you choose python as your language of choice (which I strongly recommend of course). 

For loops are a bit different (and better) in python than in Matlab. It is best explained with an example:

In [ ]:
alist = [1, 2, 'three', 4, 'end']
for element in alist:
    print(element)

In python loops, there is no index (the famous "i" in many programming languages). The "i" is not necessary because python's `for` loop is executed *over the elements* of the list. Some languages implement this syntax under the name "foreach" ("for each element in list: do something").

Note the blank spaces before "print()". In python the spacing (it's called "[indentation](http://www.python-course.eu/python3_blocks.php)") is important!

Let's study a couple of further examples:

In [ ]:
text = 'Hello!'
for letter in text:
    print(letter)

In [ ]:
for i in range(5):
    print(i)

In [ ]:
for i in range(len(text)):
    print(text[i])

Python for loops can also work like Matlab ones, with an "i". But most of the times, the "i" index is not necessary. If you think you need an "i", it is *probable* that you could also do it without it.

Now lets have a look at a 2D array:

In [ ]:
a = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
print(a)

In [ ]:
print(a.shape)

The array "a" has three rows and 4 columns. What will the following do?

In [ ]:
for row in a:
    print(row)

And the following?

In [ ]:
for row in a:
    for column in row:
        print(column)

What will the following do?

In [ ]:
for i, row in enumerate(a):
    print(i, row, a[i])

We will now compute the cumulative sum of all elements of a row in a:

In [ ]:
for i, row in enumerate(a):
    a[i] = np.cumsum(row)
a

OK, that's enough for loops for today. Let's go back to our reanaysis data.

## Read the data

Today we are going to use the upper-level atmospheric data we used last week: ``ERA-Int-MonthlyAvg-4D-UVWZ.nc``

In [ ]:
ds = xr.open_dataset('ERA-Int-MonthlyAvg-4D-UVWZ.nc')

In [ ]:
v = ds.v.mean(dim=['month', 'longitude'])
w = - ds.w.mean(dim=['month', 'longitude'])

First we are going to prepare a container for $\Psi$, where we will put the results of our computations:

In [ ]:
psi_data = v * 0  # psi_data is of the same dimensions as v: (level, latitude)

We have to integrate up to level $p$ for each level in the original data. This is why we could need a for loop. Here as an example:

In [ ]:
for p in psi_data.level.values:
    print('We will integrate from level 50 to level: {}'.format(p))

For each levels we will select only the data we need for the integration, compute it, and then store it in our container variable:

In [ ]:
for p in psi_data.level.values:
    # select parts of v we want to integrate
    sel_v = v.sel(level=slice(50, p))
    # the z coordinates of [v] for the integral:
    zcoords = sel_v.level * 100  # note the "* 100" to convert hPa into SI units!
    # integrate the v values over the atmospheric column (axis=0), with the pressure coordinates sel_v.level
    integral = np.trapz(sel_v, zcoords, axis=0)
    # store the results (".loc[p]" is equivalent to ".sel(level=p)")
    psi_data.loc[p] = integral

The loop above might be a bit complicated to understand. Don't hesitate to ask me for more details!

Finally, we need to compute the factor $ \frac{2 \pi R \cos \varphi}{g} $ and multiply it to our integral:

In [ ]:
fac = 2 * np.pi * 6371000 * np.cos(np.deg2rad(v.latitude)) / 9.81
psi = psi_data * fac
# Convert to Sverdrups:
psi = psi / 1e9
psi.name = '10$^9$ kg s$^{-1}$'

All done! Let's plot the result:

In [ ]:
psi.plot.contourf(levels=np.linspace(-100, 100, 21), extend='both')
plt.title('Stream function of the meridional overturning circulation -- annual average')
plt.ylim([1000, 50]);